In [2]:
## STEP 2A — Create Retrieval Documents

import polars as pl

# Load dataset (same as Project 4)
df = pl.read_csv(r"E:\__PROJECTS__\genai-multi-agent-project\data\raw\reviews.csv")

reviews_df = (
    df
    .select([
        pl.col("review_topic"),
        pl.col("review"),
        pl.col("score")
    ])
    .with_row_index("review_id")
    .filter(pl.col("review").is_not_null())
)

print("Total reviews:", reviews_df.shape[0])
print(reviews_df.head())





Total reviews: 2438577
shape: (5, 4)
┌───────────┬──────────────┬─────────────────────────────────┬───────┐
│ review_id ┆ review_topic ┆ review                          ┆ score │
│ ---       ┆ ---          ┆ ---                             ┆ ---   │
│ u32       ┆ str          ┆ str                             ┆ f64   │
╞═══════════╪══════════════╪═════════════════════════════════╪═══════╡
│ 0         ┆ ComputerGame ┆ Best 👌 mobile game ever        ┆ 5.0   │
│ 1         ┆ ComputerGame ┆ It's a really good game I'm go… ┆ 5.0   │
│ 2         ┆ ComputerGame ┆ Call of duty mobile is a very … ┆ 5.0   │
│ 3         ┆ ComputerGame ┆ I love the game in it's entire… ┆ 3.0   │
│ 4         ┆ ComputerGame ┆ Unplayable size.... its portab… ┆ 1.0   │
└───────────┴──────────────┴─────────────────────────────────┴───────┘


In [3]:
## STEP 2B — Convert Reviews into Retrieval Text Blocks

def build_review_document(row):
    return (
        f"Review Topic: {row['review_topic']}\n"
        f"Score: {row['score']}\n"
        f"Review Text: {row['review']}"
    )

# Sample documents (do NOT embed all 2.4M yet)
sample_docs = (
    reviews_df
    .select(["review_id", "review_topic", "score", "review"])
    .head(5000)  # start small for testing
)

documents = [
    {
        "id": int(row["review_id"]),
        "text": build_review_document(row)
    }
    for row in sample_docs.to_dicts()
]

print("Sample document:")
print(documents[0]["text"])

Sample document:
Review Topic: ComputerGame
Score: 5.0
Review Text: Best 👌 mobile game ever


## STEP 3 — VECTOR STORE CREATION

In [6]:
%pip install langchain langchain-openai chromadb

  Using cached langchain-1.2.8-py3-none-any.whl.metadata (5.0 kB)
  Using cached langchain_openai-1.1.7-py3-none-any.whl.metadata (2.6 kB)
  Using cached chromadb-1.4.1-cp39-abi3-win_amd64.whl.metadata (7.3 kB)
  Using cached langchain_core-1.2.9-py3-none-any.whl.metadata (4.4 kB)
  Using cached langgraph-1.0.7-py3-none-any.whl.metadata (7.4 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langsmith-0.6.9-py3-none-any.whl.metadata (15 kB)
  Using cached pyyaml-6.0.3-cp311-cp311-win_amd64.whl.metadata (2.4 kB)
  Using cached tenacity-9.1.3-py3-none-any.whl.metadata (1.2 kB)
  Using cached uuid_utils-0.14.0-cp39-abi3-win_amd64.whl.metadata (5.0 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached langgraph_checkpoint-4.0.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached langgraph_prebuilt-1.0.7-py3-none-any.whl.metadata (5.2 kB)
  Using cached la

In [8]:
%pip install langchain-community

  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_classic-1.0.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached sqlalchemy-2.0.46-cp311-cp311-win_amd64.whl.metadata (9.8 kB)
  Using cached aiohttp-3.13.3-cp311-cp311-win_amd64.whl.metadata (8.4 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.12.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.3-py3-none-any.whl.metadata (9.7 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached frozenlist-1.8.0-cp311-cp311-win_amd64.whl.metadata (21 kB)
  Using cached multidict-6.7.1-cp311-cp311-win_amd64.whl.metadata (5.5 kB)
  Using cached propcache-0.4.1-cp311-cp311-win_amd64.whl.metadata (14 kB)
  Using cached yarl-1.22.0-cp311-cp311-win_amd64.whl.metadata (77 kB)
  Using cached marshmallow-3.26.2-py3-none-any.whl.me

In [10]:
## STEP 3A — Create Embeddings + Vector Store

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

# Convert our documents into LangChain Documents
langchain_docs = [
    Document(
        page_content=doc["text"],
        metadata={"review_id": doc["id"]}
    )
    for doc in documents
]

print("Documents prepared for vector store:", len(langchain_docs))


Documents prepared for vector store: 5000


In [12]:
%pip install sentence-transformers

  Using cached sentence_transformers-5.2.2-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-5.1.0-py3-none-any.whl.metadata (31 kB)
  Using cached torch-2.10.0-cp311-cp311-win_amd64.whl.metadata (31 kB)
  Using cached scikit_learn-1.8.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached safetensors-0.7.0-cp38-abi3-win_amd64.whl.metadata (4.2 kB)
  Using cached networkx-3.6.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached markupsafe-3.0.3-cp311-cp311-win_amd64.whl.metadata (2.8 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached sentence_transformers-5.2.2-py3-none-any.whl (494 kB)
Using cached transformers-5.1.0-py3-none-any.whl (10.3 MB)
Using cached safetensors-0.7.0-cp38-abi3-win_amd64.whl (341 kB)
Using cached torch-2.10.0-cp311-cp311-win_amd64.whl (113.7 MB)
Using cached networkx-3.6.1-py3-none-any.wh

In [14]:
%pip install -U langchain-huggingface

  Using cached huggingface_hub-0.36.2-py3-none-any.whl.metadata (15 kB)
Using cached huggingface_hub-0.36.2-py3-none-any.whl (566 kB)

  Attempting uninstall: huggingface-hub

    Found existing installation: huggingface_hub 1.4.1

   ---------------------------------------- 0/2 [huggingface-hub]
    Uninstalling huggingface_hub-1.4.1:
   ---------------------------------------- 0/2 [huggingface-hub]
      Successfully uninstalled huggingface_hub-1.4.1
   ---------------------------------------- 0/2 [huggingface-hub]
   ---------------------------------------- 0/2 [huggingface-hub]
   ---------------------------------------- 0/2 [huggingface-hub]
   ---------------------------------------- 0/2 [huggingface-hub]
   ---------------------------------------- 0/2 [huggingface-hub]
   ---------------------------------------- 0/2 [huggingface-hub]
   ---------------------------------------- 0/2 [huggingface-hub]
   ---------------------------------------- 0/2 [huggingface-hub]
   ------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.1.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.


In [ ]:
## STEP 3B — Build the Vector Store
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2"  # Free, lightweight model
)

vectorstore = Chroma.from_documents(
    documents=langchain_docs,
    embedding=embeddings,
    collection_name="review_rag_test"
)

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 134.88it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [16]:
## STEP 3C — Test Retrieval

query = "complaints about mobile games"

results = vectorstore.similarity_search(query, k=3)

for i, res in enumerate(results, 1):
    print(f"\nResult {i}")
    print(res.page_content[:300])


Result 1
Review Topic: ComputerGame
Score: 5.0
Review Text: The best mobile game in the world

Result 2
Review Topic: ComputerGame
Score: 5.0
Review Text: The best mobile game in the world

Result 3
Review Topic: ComputerGame
Score: 5.0
Review Text: The best mobile game in the world


In [17]:
## STEP 3D — Add HARD NEGATIVE FILTERING
def build_review_document(row):
    sentiment_hint = (
        "Negative review" if row["score"] <= 2.0 else
        "Neutral review" if row["score"] <= 3.0 else
        "Positive review"
    )
    
    return (
        f"Review Type: {sentiment_hint}\n"
        f"Review Topic: {row['review_topic']}\n"
        f"Score: {row['score']}\n"
        f"Review Text: {row['review']}"
    )

In [18]:
## STEP 3E — Rebuild SAMPLE DOCUMENTS (Complaint-Focused)

# Focus on complaint-heavy reviews first
complaint_docs = (
    reviews_df
    .filter(pl.col("score") <= 2.0)
    .select(["review_id", "review_topic", "score", "review"])
    .head(3000)
)

neutral_docs = (
    reviews_df
    .filter(pl.col("score") == 3.0)
    .select(["review_id", "review_topic", "score", "review"])
    .head(1000)
)

positive_docs = (
    reviews_df
    .filter(pl.col("score") >= 4.0)
    .select(["review_id", "review_topic", "score", "review"])
    .head(1000)
)

sample_docs = pl.concat([complaint_docs, neutral_docs, positive_docs])

documents = [
    {
        "id": int(row["review_id"]),
        "text": build_review_document(row)
    }
    for row in sample_docs.to_dicts()
]

print("Total stratified documents:", len(documents))
print(documents[0]["text"])

Total stratified documents: 5000
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Unplayable size.... its portable. Update 2024 it's huge still. Make it compact


In [19]:
## STEP 3F — REBUILD VECTOR STORE (NEW COLLECTION)
'''
⚠️ Use a new collection name to avoid contamination.
'''

langchain_docs = [
    Document(
        page_content=doc["text"],
        metadata={"review_id": doc["id"]}
    )
    for doc in documents
]

vectorstore = Chroma.from_documents(
    documents=langchain_docs,
    embedding=embeddings,
    collection_name="review_rag_complaints_v1"
)

print("New vector store built")

New vector store built


In [20]:
## STEP 3G — RETEST RETRIEVAL (MOST IMPORTANT)
query = "complaints about mobile games"

results = vectorstore.similarity_search(query, k=5)

for i, res in enumerate(results, 1):
    print(f"\nResult {i}")
    print(res.page_content[:400])


Result 1
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Too many app crush

Result 2
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Too many app crush

Result 3
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Too many app crush

Result 4
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Too many app crush

Result 5
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Too many app crush


In [21]:
## STEP 3H — Split Vector Stores by Sentiment (Correct Way)

# Separate documents by sentiment
negative_docs = [
    Document(page_content=doc["text"], metadata={"review_id": doc["id"]})
    for doc in documents
    if "Negative review" in doc["text"]
]

general_docs = [
    Document(page_content=doc["text"], metadata={"review_id": doc["id"]})
    for doc in documents
]

print("Negative docs:", len(negative_docs))
print("General docs:", len(general_docs))


Negative docs: 3000
General docs: 5000


In [22]:
## create 2 vector stores

negative_vectorstore = Chroma.from_documents(
    documents=negative_docs,
    embedding=embeddings,
    collection_name="review_rag_negative_v1"
)

general_vectorstore = Chroma.from_documents(
    documents=general_docs,
    embedding=embeddings,
    collection_name="review_rag_general_v1"
)

print("Both vector stores created")


Both vector stores created


In [23]:
## STEP 3I — INTENT-AWARE RETRIEVAL (MANUAL LOGIC)

query = "complaints about mobile games"

# complaint intent detected → use negative store
results = negative_vectorstore.similarity_search(query, k=5)

for i, res in enumerate(results, 1):
    print(f"\nResult {i}")
    print(res.page_content[:400])


Result 1
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Too many app crush

Result 2
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Too many app crush

Result 3
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Too many app crush

Result 4
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Too many app crush

Result 5
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Too many app crush


## STEP 4 — CONVERSATIONAL ORCHESTRATION

In [24]:
%pip install autogen

  Using cached autogen-0.10.5-py3-none-any.whl.metadata (24 kB)
  Using cached ag2-0.10.5-py3-none-any.whl.metadata (36 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached termcolor-3.3.0-py3-none-any.whl.metadata (6.5 kB)
Using cached autogen-0.10.5-py3-none-any.whl (13 kB)
Using cached ag2-0.10.5-py3-none-any.whl (962 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
Using cached docker-7.1.0-py3-none-any.whl (147 kB)
Using cached termcolor-3.3.0-py3-none-any.whl (7.7 kB)

   ---------------- ----------------------- 2/5 [docker]
   ---------------- ----------------------- 2/5 [docker]
   ---------------- ----------------------- 2/5 [docker]
   ---------------- ----------------------- 2/5 [docker]
   ------------------------ --------------- 3/5 [ag2]
   ------------------------ --------------- 3/5 [ag2]
   ------------------------ --------------- 3/5 [ag2]
   ------------------------

In [28]:
%pip install autogen-agentchat autogen-ext sentence-transformers chromadb

  Using cached huggingface_hub-1.4.1-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-1.4.1-py3-none-any.whl (553 kB)
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/7.0 MB 1.4 MB/s eta 0:00:05
   ----- ---------------------------------- 1.0/7.0 MB 1.7 MB/s eta 0:00:04
   ------- -------------------------------- 1.3/7.0 MB 1.6 MB/s eta 0:00:04
   -------- ------------------------------- 1.6/7.0 MB 1.5 MB/s eta 0:00:04
   ---------- ----------------------------- 1.8/7.0 MB 1.4 MB/s eta 0:00:04
   ----------- ---------------------------- 2.1/7.0 MB 1.4 MB/s eta 0:00:04
   -------------- ------------------------- 2.6/7.0 MB 1.6 MB/s eta 0:00:03
   ----------------- ---------------------- 3.1/7.0 MB 1.7 MB/s eta 0:00:03
   -------------------- ------------------- 3.7/7.0 MB 1.8 MB/s eta 0:00:02
   ----------------------- ---------------- 4.

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.4.1 which is incompatible.


In [30]:
import os
import asyncio
from dotenv import load_dotenv
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

# Load environment variables
load_dotenv()
groq_api_key = os.getenv('GROQCLOUD_API_KEY')

if not groq_api_key:
    raise ValueError("GROQCLOUD_API_KEY not found in .env file")

In [31]:
groq_model_info = {
    "vision": False,
    "function_calling": True,
    "json_output": True,
    "family": "unknown",
    "structured_output": False
}

model_client = OpenAIChatCompletionClient(
    model="llama-3.3-70b-versatile",
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1",
    model_info=groq_model_info
)

In [38]:
def retrieve_reviews(query: str, top_k: int = 8) -> str:
    """Retrieve relevant reviews with deduplication"""
    results = negative_vectorstore.similarity_search(query, k=top_k * 2)

    seen = set()
    unique_reviews = []

    for doc in results:
        text = doc.page_content.strip()
        if text not in seen:
            seen.add(text)
            unique_reviews.append(text)
        if len(unique_reviews) >= top_k:
            break

    formatted = "\n\n---\n\n".join([
        f"Review {i+1}:\n{text}"
        for i, text in enumerate(unique_reviews)
    ])

    return (
        f"Found {len(unique_reviews)} unique complaint reviews "
        f"(deduplicated from {len(results)} retrieved):\n\n{formatted}"
    )


In [46]:
## STEP 5: Define Agents

retriever_agent = AssistantAgent(
    name="Retriever",
    model_client=model_client,
    system_message="""You are a review retrieval specialist.
    Use the retrieve_reviews tool to find relevant customer reviews.
    Present the reviews clearly without analyzing them.""",
    tools=[retrieve_reviews]
)

analyzer_agent = AssistantAgent(
    name="Analyzer",
    model_client=model_client,
    system_message="""You are a sentiment analysis expert.
    Analyze the reviews provided by the Retriever.
    Identify:
    - Overall sentiment (positive/negative/neutral)
    - Key themes and patterns
    - Specific pain points
    - Severity of issues"""
)

categorizer_agent = AssistantAgent(
    name="Categorizer",
    model_client=model_client,
    system_message="""You categorize customer complaints into themes:
    - Performance Issues (lag, crashes, freezes)
    - Monetization (ads, pay-to-win, pricing)
    - Technical Problems (bugs, battery drain)
    - UI/UX Problems (confusing interface, navigation)
    - Customer Service
    
    Provide counts and priority levels for each category."""
)

summarizer_agent = AssistantAgent(
    name="Summarizer",
    model_client=model_client,
    system_message="""You create executive summaries with:
    - Top 3 critical issues
    - Overall sentiment breakdown
    - Actionable recommendations
    - Priority rankings"""
)

decision_agent = AssistantAgent(
    name="DecisionMaker",
    model_client=model_client,
    system_message="""
                    You are a Lead Business Analyst.

                    Your task is to produce the FINAL output of the analysis
                    in STRICT JSON format.

                    Rules (VERY IMPORTANT):
                    - Output ONLY valid JSON
                    - Do NOT include explanations outside JSON
                    - Do NOT repeat agent conversations
                    - Base all fields ONLY on retrieved reviews
                    - If evidence is limited, reflect that honestly

                    Required JSON schema:

                    {
                    "topic": "<review topic>",
                    "analysis_scope": "<e.g. Negative reviews only>",
                    "evidence_count": <number>,
                    "confidence_level": "<High | Medium | Low>",
                    "primary_issue": "<main problem identified>",
                    "evidence": [
                        "<short quote or paraphrase from review>"
                    ],
                    "root_causes": [
                        "<cause 1>",
                        "<cause 2>"
                    ],
                    "business_impact": [
                        "<impact 1>",
                        "<impact 2>"
                    ],
                    "recommended_actions": [
                        "<action 1>",
                        "<action 2>"
                    ]
                    }

                    End output immediately after JSON.
                    """
)



team = RoundRobinGroupChat(
    participants=[
    retriever_agent,
    analyzer_agent,
    categorizer_agent,
    decision_agent
]
,
    termination_condition=MaxMessageTermination(max_messages=7),
)

In [47]:
# STEP 7: Run Analysis

async def run_analysis(query: str):
    """Run the multi-agent analysis"""
    print(f"\n{'='*80}")
    print(f"QUERY: {query}")
    print(f"{'='*80}\n")
    
    result = await team.run(task=query)
    
    print(f"\n{'='*80}")
    print("ANALYSIS RESULTS")
    print(f"{'='*80}\n")
    
    for i, msg in enumerate(result.messages, 1):
        print(f"\n[Message {i}] Agent: {msg.source}")
        print("-" * 80)
        print(msg.content)
        print()
    
    return result

In [48]:
queries ="""
        Retrieve relevant customer reviews about mobile game complaints.
        Analyze them to identify recurring issues.
        Categorize the issues.
        Then produce ONE final business summary with recommendations.
        """
    
# Run first query
result = await run_analysis(queries[0])


QUERY: 



ANALYSIS RESULTS


[Message 1] Agent: user
--------------------------------------------------------------------------------




[Message 2] Agent: Retriever
--------------------------------------------------------------------------------
[FunctionCall(id='d2ma1ckdx', arguments='{"query":"customer reviews","top_k":8}', name='retrieve_reviews')]


[Message 3] Agent: Retriever
--------------------------------------------------------------------------------
[FunctionExecutionResult(content='Found 1 unique complaint reviews (deduplicated from 16 retrieved):\n\nReview 1:\nReview Type: Negative review\nReview Topic: ComputerGame\nScore: 1.0\nReview Text: This game is very nice but the reason why I gave it 1 star is because it keeps freezing and kicks me out of the game for example when I play a clan game it kicks me out in the middle of the game and all my kills is gone there for no clan points.', name='retrieve_reviews', call_id='d2ma1ckdx', is_error=False)]


[Message 4] Agent: 